# Poke Platform DB Exploration

This notebook connects directly to RDS using Secrets Manager and loads results into pandas DataFrames.

Set `REGION` and `SECRET_ARN` as environment variables if you want to override defaults.


In [1]:
# %pip install pandas
%pip install boto3 pandas psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import sys
from pathlib import Path

# Ensure repo root is on sys.path for local imports
repo_root = Path().resolve().parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

import pandas as pd

from scripts.db_notebook_helpers import fetch_db_secret, read_sql

REGION = os.environ.get("REGION", "us-east-2")
SECRET_ARN = os.environ.get("SECRET_ARN", "arn:aws:secretsmanager:us-east-2:878817878019:secret:PokePlatformStackPostgresSe-uD7oKVjyUmYi-NFBVMe")


## Quick sanity checks


In [2]:
read_sql("SELECT COUNT(*) AS active_tracked FROM tracked_asset WHERE is_active=true;", region=REGION, secret_arn=SECRET_ARN)


/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,active_tracked
0,6131
1,Exiting session with sessionId: ecs-execute-co...


In [4]:
df_cost = read_sql("SELECT * FROM cardmarket_price_snapshot limit 10;", region=REGION, secret_arn=SECRET_ARN)
df_cost.head()

/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,snapshot_date,snapshot_ts,asset_id,variant,currency,avg1,avg7,avg30,low_price,trend_price,url,source_updated_at,extra
0,2026-01-19,2026-01-19 13:00:51.662972+00,ptcg:sm6-1,default,EUR,0.12,0.16,0.13,0.02,0.16,https://prices.pokemontcg.io/cardmarket/sm6-1,2026/01/19,"{""avg1"": 0.12, ""avg7"": 0.16, ""avg30"": 0.13, ""l..."
1,2026-01-19,2026-01-19 13:00:51.662972+00,ptcg:sm6-1,reverseHolo,EUR,0.50,0.41,0.33,NaN,0.36,https://prices.pokemontcg.io/cardmarket/sm6-1,2026/01/19,"{""reverseHoloAvg1"": 0.5, ""reverseHoloAvg7"": 0...."
2,2026-01-19,2026-01-19 13:00:51.662972+00,ptcg:sm6-2,default,EUR,0.53,0.58,0.91,0.10,0.26,https://prices.pokemontcg.io/cardmarket/sm6-2,2026/01/19,"{""avg1"": 0.53, ""avg7"": 0.58, ""avg30"": 0.91, ""l..."
3,2026-01-19,2026-01-19 13:00:51.662972+00,ptcg:sm6-2,reverseHolo,EUR,0.35,0.83,0.74,NaN,0.80,https://prices.pokemontcg.io/cardmarket/sm6-2,2026/01/19,"{""reverseHoloAvg1"": 0.35, ""reverseHoloAvg7"": 0..."
4,2026-01-19,2026-01-19 13:00:51.662972+00,ptcg:sm6-2a,default,EUR,1.00,1.73,1.31,0.15,1.93,https://prices.pokemontcg.io/cardmarket/sm6-2a,2026/01/19,"{""avg1"": 1.0, ""avg7"": 1.73, ""avg30"": 1.31, ""lo..."


In [ ]:
df_card_metadata = read_sql("SELECT images_json FROM card_metadata where name = 'Mega Charizard X ex';", region=REGION, secret_arn=SECRET_ARN)
df_card_metadata

/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


KeyError: 0

In [4]:
read_sql("SELECT COUNT(*) AS card_metadata_count FROM card_metadata;", region=REGION, secret_arn=SECRET_ARN)


/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,card_metadata_count
0,6131
1,Exiting session with sessionId: ecs-execute-co...


In [5]:
read_sql("SELECT MAX(updated_ts) AS latest_updated_ts FROM card_metadata;", region=REGION, secret_arn=SECRET_ARN)


/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,latest_updated_ts
0,2026-01-23 13:16:31.781371+00
1,Exiting session with sessionId: ecs-execute-co...


## Sample slice from card_metadata


In [6]:
df_cards = read_sql("SELECT asset_id, name, set_name, rarity, updated_ts FROM card_metadata ORDER BY updated_ts DESC LIMIT 50;", region=REGION, secret_arn=SECRET_ARN)
df_cards.head(10)


/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,asset_id,name,set_name,rarity,updated_ts
0,ptcg:sm8-32,Vespiquen,Lost Thunder,Uncommon,2026-01-23 13:16:31.781371+00
1,ptcg:sm8-51,Pyroar,Lost Thunder,Rare,2026-01-23 13:16:31.781371+00
2,ptcg:sm8-16,Shuckle,Lost Thunder,Uncommon,2026-01-23 13:16:31.781371+00
3,ptcg:sm8-31,Combee,Lost Thunder,Common,2026-01-23 13:16:31.781371+00
4,ptcg:sm8-41,Quilava,Lost Thunder,Uncommon,2026-01-23 13:16:31.781371+00
5,ptcg:sm8-50,Litleo,Lost Thunder,Common,2026-01-23 13:16:31.781371+00
6,ptcg:sm8-8,Meganium,Lost Thunder,Rare Holo,2026-01-23 13:16:31.781371+00
7,ptcg:sm8-15,Pineco,Lost Thunder,Common,2026-01-23 13:16:31.781371+00
8,ptcg:sm8-23,Wurmple,Lost Thunder,Common,2026-01-23 13:16:31.781371+00
9,ptcg:sm8-30,Ninjask,Lost Thunder,Uncommon,2026-01-23 13:16:31.781371+00


In [9]:
df_car2= read_sql("SELECT * from cards limit 10;", region=REGION, secret_arn=SECRET_ARN)
df_car2.head(10)

/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,"psql:/tmp/query.sql:1: ERROR: relation ""cards"" does not exist"
0,LINE 1: COPY ( SELECT * from cards limit 10 )...
1,^
2,Exiting session with sessionId: ecs-execute-co...


In [ ]:
tables

## Your custom query


In [ ]:
custom_sql = """
SELECT *
FROM tracked_asset
WHERE is_active=true
LIMIT 50;
"""
df_custom = read_sql(custom_sql, region=REGION, secret_arn=SECRET_ARN)
df_custom
